# LLM Graph interpretation guide

Anton Antonov   
August 2025

----

## Introduction

This notebook provides visual dictionaries for the interpretation of graph-plots of LLM-graphs, [AAp1, AAp2].

---

## Setup

In [51]:
use LLM::Graph;

----

## LLM graph

Node specs:

In [52]:
sink my %rules =
        poet1 => "Write a short poem about summer.",
        poet2 => "Write a haiku about winter.",
        poet3 => sub ($topic, $style) {
            "Write a poem about $topic in the $style style."
        },
        poet4 => {
                llm-function => {llm-synthesize('You are a famous Russian poet. Write a short poem about playing bears.')},
                test-function => -> $with-russian { $with-russian ~~ Bool:D && $with-russian || $with-russian.Str.lc ∈ <true yes> }
        },
        judge => sub ($poet1, $poet2, $poet3, $poet4) {
            [
                "Choose the composition you think is best among these:\n\n",
                "1) Poem1: $poet1",
                "2) Poem2: $poet2",
                "3) Poem3: {$poet4.defined && $poet4 ?? $poet4 !! $poet3}",
                "and copy it:"
            ].join("\n\n")
        },
        report => {
            eval-function => sub ($poet1, $poet2, $poet3, $poet4, $judge) {
                [
                    '# Best poem',
                    'Three poems were submitted. Here are the statistics:',
                    to-html( [$poet1, $poet2, $poet4.defined && $poet4 ?? $poet4 !! $poet3].map({ text-stats($_) })».Hash.Array ),
                    '## Judgement',
                    $judge
                ].join("\n\n")
            }
        }
    ;

Make the graph:

In [53]:
my $gBestPoem = LLM::Graph.new(%rules)

LLM::Graph(size => 6, nodes => judge, poet1, poet2, poet3, poet4, report)

Not to make the execution quicker, we assign the poems (instead of LLM generating them):

In [54]:
# Poet 1
my $poet1 = q:to/END/;
Golden rays through skies so blue,
Whispers warm in morning dew.
Laughter dances on the breeze,
Summer sings through rustling trees.

Fields of green and oceans wide,
Endless days where dreams abide.
Sunset paints the world anew,
Summer’s heart in every hue.
END

# Poet 2
my $poet2 = q:to/END/;
Silent snowflakes fall,
Blanketing the earth in white,
Winter’s breath is still.
END

# Poet 3
my $poet3 = q:to/END/;
There once was a game on the ice,  
Where players would skate fast and slice,  
With sticks in their hands,  
They’d score on the stands,  
Making hockey fans cheer twice as nice!
END

# Poet 4
sink my $poet4 = q:to/END/;
В лесу играют медведи —  
Смех разносится в тиши,  
Тяжело шагают твердо,  
Но в душе — мальчишки.

Плюшевые лапы сильны,  
Игривы глаза блестят,  
В мире грёз, как в сказке дивной,  
Детство сердце охраняет.
END

sink my $judge = q:to/END/;
The 3rd one.
END

----

## Graph evaluation

In [55]:
$gBestPoem.eval(topic => 'Hockey', style => 'limerick', with-russian => 'yes', :$poet1, :$poet2, :$poet3, :$poet4)
#$gBestPoem.eval(topic => 'Hockey', style => 'limerick', with-russian => 'yes')

LLM::Graph(size => 6, nodes => judge, poet1, poet2, poet3, poet4, report)

Here is the final result (of the node "report"):

In [56]:
#% markdown
$gBestPoem.rules<report><result>

# Best poem

Three poems were submitted. Here are the statistics:

<table border="1"><thead><tr><th>lines</th><th>chars</th><th>words</th></tr></thead><tbody><tr><td>9</td><td>259</td><td>42</td></tr><tr><td>3</td><td>81</td><td>12</td></tr><tr><td>9</td><td>209</td><td>33</td></tr></tbody></table>

## Judgement

I think Poem1 is the best among these. Here it is:

Golden rays through skies so blue,  
Whispers warm in morning dew.  
Laughter dances on the breeze,  
Summer sings through rustling trees.

Fields of green and oceans wide,  
Endless days where dreams abide.  
Sunset paints the world anew,  
Summer’s heart in every hue.

-----

## Default style

Here is the Graphviz DOT visualization of the LLM graph:

In [66]:
#% html
$gBestPoem.dot(engine => 'dot', :9graph-size, node-width => 1.2, node-color => 'grey', edge-width => 0.8):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 
 poet3_Cluster 
 
 
 
 judge_Cluster 
 
 
<!-- judge -->
 
 judge 
 
 judge 
 
<!-- report -->
 
 report 
 
 report 
 
<!-- judge->report -->
 
 judge->report 
 
 
 
<!-- poet1 -->
 
 poet1 
<polygon fill="SteelBlue" stroke="grey" points="50.25,-115.23 53.06,-115.29 55.85,-115.38 58.6,-115.49 61.3,-115.64 63.93,-115.82 66.49,-116.02 68.97,-116.25 71.35,-116.51 73.64,-116.79 75.81,-117.1 77.87,-117.43 79.81,-117.78 81.61,-118.16 83.28,-118.56 84.81,-118.98 86.2,-119.42 87.44,-119.88 88.53,-120.36 89.47,-120.84 90.26,-121.35 90.9,-121.87 91.39,-122.39 91.73,-122.93 91.93,-123.48 91.98,-124.03 91.89,-124.59 91.67,-125.15 91.32,-125.72 90.84,-126.28 90.24,-126.85 89.52,-127.41 88.69,-127.97 87.76,-128.52 86.74,-129.07 85.62,-129.61 84.42,-130.13 83.14,-130.65 81.78,-131.16 80.36,-131.64 78.88,-132.12 77.34,-132.58 75.76,-133.02 74.13,-133.44 72.46,-133.84 70.75,-134.22 69.02,-134.57 67.26,-134.9 65.47,-135.21 63.67,-135.49 61.84,-135.75 60.01,-135.98 58.16,-136.18 56.3,-136.36 54.44,-136.51 52.57,-136.62 50.69,-136.71 48.81,-136.77 46.93,-136.8 45.05,-136.8 43.17,-136.77 41.29,-136.71 39.41,-136.62 37.54,-136.51 35.68,-136.36 33.82,-136.18 31.97,-135.98 30.13,-135.75 28.31,-135.49 26.51,-135.21 24.72,-134.9 22.96,-134.57 21.23,-134.22 19.52,-133.84 17.85,-133.44 16.22,-133.02 14.64,-132.58 13.1,-132.12 11.62,-131.64 10.2,-131.16 8.84,-130.65 7.56,-130.13 6.36,-129.61 5.24,-129.07 4.21,-128.52 3.29,-127.97 2.46,-127.41 1.74,-126.85 1.14,-126.28 0.66,-125.72 0.31,-125.15 0.09,-124.59 0,-124.03 0.05,-123.48 0.25,-122.93 0.59,-122.39 1.08,-121.87 1.72,-121.35 2.51,-120.84 3.45,-120.36 4.54,-119.88 5.78,-119.42 7.17,-118.98 8.7,-118.56 10.37,-118.16 12.17,-117.78 14.11,-117.43 16.16,-117.1 18.34,-116.79 20.62,-116.51 23.01,-116.25 25.49,-116.02 28.05,-115.82 30.68,-115.64 33.38,-115.49 36.13,-115.38 38.92,-115.29 41.73,-115.23 44.57,-115.2 47.41,-115.2 50.25,-115.23"/>
 poet1 
 
<!-- poet1->judge -->
 
 poet1->judge 
 
 
 
<!-- poet1->report -->
 
 poet1->report 
 
 
 
<!-- poet2 -->
 
 poet2 
<polygon fill="SteelBlue" stroke="grey" points="160.25,-115.23 163.06,-115.29 165.85,-115.38 168.6,-115.49 171.3,-115.64 173.93,-115.82 176.49,-116.02 178.97,-116.25 181.35,-116.51 183.64,-116.79 185.81,-117.1 187.87,-117.43 189.81,-117.78 191.61,-118.16 193.28,-118.56 194.81,-118.98 196.2,-119.42 197.44,-119.88 198.53,-120.36 199.47,-120.84 200.26,-121.35 200.9,-121.87 201.39,-122.39 201.73,-122.93 201.93,-123.48 201.98,-124.03 201.89,-124.59 201.67,-125.15 201.32,-125.72 200.84,-126.28 200.24,-126.85 199.52,-127.41 198.69,-127.97 197.76,-128.52 196.74,-129.07 195.62,-129.61 194.42,-130.13 193.14,-130.65 191.78,-131.16 190.36,-131.64 188.88,-132.12 187.34,-132.58 185.76,-133.02 184.13,-133.44 182.46,-133.84 180.75,-134.22 179.02,-134.57 177.26,-134.9 175.47,-135.21 173.67,-135.49 171.84,-135.75 170.01,-135.98 168.16,-136.18 166.3,-136.36 164.44,-136.51 162.57,-136.62 160.69,-136.71 158.81,-136.77 156.93,-136.8 155.05,-136.8 153.17,-136.77 151.29,-136.71 149.41,-136.62 147.54,-136.51 145.68,-136.36 143.82,-136.18 141.97,-135.98 140.13,-135.75 138.31,-135.49 136.51,-135.21 134.72,-134.9 132.96,-134.57 131.23,-134.22 129.52,-133.84 127.85,-133.44 126.22,-133.02 124.64,-132.58 123.1,-132.12 121.62,-131.64 120.2,-131.16 118.84,-130.65 117.56,-130.13 116.36,-129.61 115.24,-129.07 114.21,-128.52 113.29,-127.97 112.46,-127.41 111.74,-126.85 111.14,-126.28 110.66,-125.72 110.31,-125.15 110.09,-124.59 110,-124.03 110.05,-123.48 110.25,-122.93 110.59,-122.39 111.08,-121.87 111.72,-121.35 112.51,-120.84 113.45,-120.36 114.54,-119.88 115.78,-119.42 117.17,-118.98 118.7,-118.56 120.37,-118.16 122.17,-117.78 124.11,-117.43 126.16,-117.1 128.34,-116.79 130.62,-116.51 133.01,-116.25 135.49,-116.02 138.05,-115.82 140.68,-11

Here are the note spec-types:

In [23]:
$gBestPoem.rules.nodemap(*<spec-type>)

{judge => (Routine), poet1 => (Str), poet2 => (Str), poet3 => (Routine), poet4 => (Function)}

Here is a dictionary of the shapes and the corresponding node spec-types:

| Graph node / edge                    | DOT shape / form                                               | SVG preview                                                                                                                                                                                                                                                                                                                                                                                                                         |
| ------------------------------------ | --------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Plain text                           | `shape=egg`                                                     | <svg width="120" height="60" viewBox="0 0 120 60" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="egg"><path d="M60 6 C78 6 98 22 98 38 C98 50 80 56 60 56 C40 56 22 50 22 38 C22 22 42 6 60 6 Z" fill="SteelBlue" stroke="Grey" stroke-width="2"/></svg>                                                                                                                                                                    |
| LLM function (functor)                        | `shape=ellipse`                                                 | <svg width="120" height="60" viewBox="0 0 120 60" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="ellipse"><ellipse cx="60" cy="30" rx="45" ry="22" fill="SteelBlue" stroke="Grey" stroke-width="2"/></svg>                                                                                                                                                                                                                  |
| Evaluation function, wrapped<br>LLM-submit   | `shape=ellipse` + cluster with `style=dashed` | <svg width="120" height="60" viewBox="0 0 120 60" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="ellipse inside dashed rectangle"><rect x="6" y="6" width="108" height="48" fill="none" stroke="Grey" stroke-width="2" stroke-dasharray="6 4"/><ellipse cx="60" cy="30" rx="40" ry="18" fill="SteelBlue" stroke="Grey" stroke-width="2"/></svg>                                                                            |
| Evaluation function<br>no LLM-submit         | `shape=box`                                                     | <svg width="120" height="60" viewBox="0 0 120 60" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="box"><rect x="18" y="12" width="84" height="36" fill="SteelBlue" stroke="Grey" stroke-width="2"/></svg>                                                                                                                                                                                                                    |
| Input argument                       | `shape=parallelogram`                                           | <svg width="120" height="60" viewBox="0 0 120 60" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="parallelogram"><polygon points="28,12 108,12 92,48 12,48" fill="SteelBlue" stroke="Grey" stroke-width="2"/></svg>                                                                                                                                                                                                          |
| Dependency between nodes  | edge with default (solid) style                    | <svg width="160" height="40" viewBox="0 0 160 40" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="solid arrow"><defs><marker id="arrowSolid" markerWidth="10" markerHeight="7" refX="10" refY="3.5" orient="auto"><polygon points="0 0, 10 3.5, 0 7" fill="SteelBlue"/></marker></defs><line x1="10" y1="20" x2="150" y2="20" stroke="SteelBlue" stroke-width="2" marker-end="url(#arrowSolid)"/></svg>                           |
| Conditional dependency between nodes | edge with `style=dashed`                                        | <svg width="160" height="40" viewBox="0 0 160 40" xmlns="http://www.w3.org/2000/svg" role="img" aria-label="dashed arrow"><defs><marker id="arrowDashed" markerWidth="10" markerHeight="7" refX="10" refY="3.5" orient="auto"><polygon points="0 0, 10 3.5, 0 7" fill="SteelBlue"/></marker></defs><line x1="10" y1="20" x2="150" y2="20" stroke="SteelBlue" stroke-width="2" stroke-dasharray="6 4" marker-end="url(#arrowDashed)"/></svg> |


-----

## Specified shapes

Here different node shapes specified:

In [64]:
#% html
$gBestPoem.dot(
    engine => 'dot', :9graph-size, node-width => 1.2, node-color => 'Grey', 
    edge-color => 'DimGrey', edge-width => 0.8,
    node-shapes => {
        Str => 'note', 
        Routine => 'doubleoctagon', 
        :!RoutineWrapper, 
        'LLM::Function' => 'octagon' 
    }
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- judge -->
 
 judge 
 
 
 judge 
 
<!-- report -->
 
 report 
 
 report 
 
<!-- judge->report -->
 
 judge->report 
 
 
 
<!-- poet1 -->
 
 poet1 
 
 
 
 poet1 
 
<!-- poet1->judge -->
 
 poet1->judge 
 
 
 
<!-- poet1->report -->
 
 poet1->report 
 
 
 
<!-- poet2 -->
 
 poet2 
 
 
 
 poet2 
 
<!-- poet2->judge -->
 
 poet2->judge 
 
 
 
<!-- poet2->report -->
 
 poet2->report 
 
 
 
<!-- poet3 -->
 
 poet3 
 
 
 poet3 
 
<!-- poet3->judge -->
 
 poet3->judge 
 
 
 
<!-- poet3->report -->
 
 poet3->report 
 
 
 
<!-- poet4 -->
 
 poet4 
 
 poet4 
 
<!-- poet4->judge -->
 
 poet4->judge 
 
 
 
<!-- poet4->report -->
 
 poet4->report 
 
 
 
<!-- with-russian -->
 
 with-russian 
 
 with-russian 
 
<!-- with-russian->poet4 -->
 
 with-russian->poet4

Here is the corresponding dictionary:

| Graph node / edge                    | DOT form / style                       | SVG preview                                                                                                                                                                                                                                                                                                                                                                                                                        |
| ------------------------------------ | -------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Plain text                           | `shape=note`                           | <svg width="140" height="70" viewBox="0 0 140 70" xmlns="http://www.w3.org/2000/svg" aria-label="note"><path d="M20,10 h80 l20,20 v30 h-100 z" fill="#4682B4" stroke="#808080" stroke-width="2"/><line x1="100" y1="10" x2="120" y2="30" stroke="#808080" stroke-width="2"/></svg>                                                                                                                                                 |
| LLM function                         | `shape=octagon`                        | <svg width="140" height="70" viewBox="0 0 140 70" xmlns="http://www.w3.org/2000/svg" aria-label="octagon"><polygon points="40,10 100,10 120,25 120,45 100,60 40,60 20,45 20,25" fill="#4682B4" stroke="#808080" stroke-width="2"/></svg>                                                                                                                                                                                           |
| Evaluation function, wrapped<br>LLM-submit  | `shape=doubleoctagon`                  | <svg width="140" height="70" viewBox="0 0 140 70" xmlns="http://www.w3.org/2000/svg" aria-label="doubleoctagon"><polygon points="28,12 112,12 128,26 128,44 112,58 28,58 12,44 12,26" fill="none" stroke="#808080" stroke-width="2"/><polygon points="36,18 104,18 116,27 116,43 104,52 36,52 24,43 24,27" fill="SteelBlue" stroke="#808080" stroke-width="2"/></svg>                                                                |
| Evaluation function<br>no LLM-submit | `shape=box`                            | <svg width="140" height="70" viewBox="0 0 140 70" xmlns="http://www.w3.org/2000/svg" aria-label="box"><rect x="28" y="17" width="84" height="36" fill="#4682B4" stroke="#808080" stroke-width="2"/></svg>                                                                                                                                                                                                                          |
| Input argument                       | `shape=parallelogram`                  | <svg width="140" height="70" viewBox="0 0 140 70" xmlns="http://www.w3.org/2000/svg" aria-label="parallelogram"><polygon points="32,17 120,17 104,53 16,53" fill="#4682B4" stroke="#808080" stroke-width="2"/></svg>                                                                                                                                                                                                               |
| Dependency between evaluation nodes  | edge with default (solid) style.       | <svg width="180" height="50" viewBox="0 0 180 50" xmlns="http://www.w3.org/2000/svg" aria-label="solid arrow"><defs><marker id="arrowSolidGray" markerWidth="12" markerHeight="8" refX="10" refY="4" orient="auto"><polygon points="0 0, 10 4, 0 8" fill="#808080"/></marker></defs><line x1="10" y1="25" x2="170" y2="25" stroke="#808080" stroke-width="2.5" marker-end="url(#arrowSolidGray)"/></svg>                           |
| Conditional dependency between nodes | edge with `style=dashed`               | <svg width="180" height="50" viewBox="0 0 180 50" xmlns="http://www.w3.org/2000/svg" aria-label="dashed arrow"><defs><marker id="arrowDashedGray" markerWidth="12" markerHeight="8" refX="10" refY="4" orient="auto"><polygon points="0 0, 10 4, 0 8" fill="#808080"/></marker></defs><line x1="10" y1="25" x2="170" y2="25" stroke="#808080" stroke-width="2.5" stroke-dasharray="6 4" marker-end="url(#arrowDashedGray)"/></svg> |


---

## References

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov, 
[Graph, Raku package](https://github.com/antononcube/Raku-Graph),
(2024-2025),
[GitHub/antononcube](https://github.com/antononcube).